# Installing packages and loading libraries

In [1]:
import os
import ast
import sys
import json
import random
import logging
import argparse
import rasterio
import numpy as np
from time import time 
from tqdm import tqdm 
from glob import glob
from os.path import dirname as up
from rasterio.enums import Resampling
import torchvision.transforms as transforms

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.nn import functional as F
from timm.utils import ModelEma

import segmentation_models_pytorch as smp
from vscp import VSCP
from test_time_aug import TTA
from metrics import Evaluation, confusion_matrix
from assets import bool_flag, cosine_scheduler, labels, mados_cat_mapping, mados_color_mapping
#from dataset import MADOS, gen_weights, class_distr, bands_mean, bands_std
from dataset_crop import MADOS, gen_weights, class_distr, bands_mean, bands_std
import torch.nn as nn
import pickle

torch.use_deterministic_algorithms(True)

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
log_directory = os.path.join("/home/ubuntu/Tesi/Early-Fusion-Unet", 'log')
time_now = str(int(time() / 60))
log_file = os.path.join(log_directory, 'log_marinext_' + time_now + '.log')

logging.basicConfig(filename=log_file, filemode='a', level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')
logging.info('*' * 10)

# Seed

In [3]:
def seed_all(seed):
    # Pytorch Reproducibility
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
def seed_worker(worker_id):
    # DataLoader Workers Reproducibility
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# Data

### Reproducibility

In [4]:
seed_all(0)
g=torch.Generator()
g.manual_seed(0)

writer = SummaryWriter(os.path.join(log_directory, 'logs', 'tsboard_segm'+'_'+time_now))

2024-07-24 09:31:05.165731: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 09:31:10.997066: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Construct Data loader

In [5]:
splits_path = os.path.join("/home/ubuntu/Tesi/MADOS_crop/MADOS",'splits')

In [6]:
dataset_train = MADOS("/home/ubuntu/Tesi/MADOS_crop/MADOS", splits_path, 'train')
dataset_val = MADOS("/home/ubuntu/Tesi/MADOS_crop/MADOS", splits_path, 'val')
dataset_test = MADOS("/home/ubuntu/Tesi/MADOS_crop/MADOS", splits_path, 'test')

Load train set to memory:   0%|          | 0/175 [00:00<?, ?it/s]/home/ubuntu/.local/lib/python3.8/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
Load test set to memory: 100%|██████████| 175/175 [01:15<00:00,  2.33it/s]


In [7]:
train_loader = DataLoader(dataset_train, 
                          batch_size = 5, 
                          shuffle = True,
                          num_workers = 0,           # 0 is the main process
                          pin_memory = False,        # Use pinned memory or not
                          prefetch_factor = 2,       # Number of sample loaded in advance by each worker
                          persistent_workers= False, # This allows to maintain the workers Dataset instances alive.
                          worker_init_fn=seed_worker,
                          generator=g,
                          drop_last=True)

In [8]:
val_loader = DataLoader(dataset_val, 
                          batch_size = 5, 
                          shuffle = False,
                          num_workers = 0,           # 0 is the main process
                          pin_memory = False,        # Use pinned memory or not
                          prefetch_factor = 2,       # Number of sample loaded in advance by each worker
                          persistent_workers= False, # This allows to maintain the workers Dataset instances alive.
                          worker_init_fn=seed_worker,
                          generator=g)

In [9]:
test_loader = DataLoader(dataset_test, 
                          batch_size = 1, 
                          shuffle = False)

### GPU presence check

In [10]:
# Use gpu or cpu
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [11]:
model = smp.Unet('resnet18', in_channels=11, classes=15)
model.to(device)

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(11, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [12]:
from torchsummary import summary
summary(model, (11, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]          34,496
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

# Training

EMA è una forma di media mobile che attribuisce pesi decrescenti ai dati nel tempo, con un peso maggiore ai dati più recenti.

In [12]:
model_ema = None
if True:
    model_ema = ModelEma(
        model,
        decay=0.999,
        device=device,
        resume='')
    
    ema_decay_schedule = cosine_scheduler(0.999, 0.999, 80, len(train_loader))

Weighted Cross Entropy Loss & adam optimizer

In [13]:
weight = torch.tensor(np.array([64.39, 28.33, 70.90, 75.34, 75.33, 19.25, 5.67, 26.99, 53.84, 23.63, 15.65, 75.48, 75.03, 76.19, 20.38])).float()
criterion = torch.nn.CrossEntropyLoss(reduction= 'none', weight=weight.to(device))
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002, weight_decay=0)

Learning Rate scheduler

In [14]:
reduce_lr_on_plateau=0
if reduce_lr_on_plateau==1:
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
else:
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, '[45,65]', gamma=0.1)

In [15]:
epochs = 80
eval_every = 1

### Start training

In [16]:
#Train
model.train()

best_val_loss = 100
best_val_loss_ema = 100

for epoch in range(1, epochs+1):

    training_loss = []
    training_batches = 0
    
    i_board = 0
    for it, (image, target) in enumerate(tqdm(train_loader, desc="training")):
        
        it = len(train_loader) * (epoch-1) + it  # global training iteration
        
        #VSCP
        image_augmented, target_augmented = VSCP(image.cpu().numpy(), target.cpu().numpy())
        image = torch.cat([image, torch.tensor(image_augmented).to(image.device)])
        target = torch.cat([target, torch.tensor(target_augmented).to(target.device)])

        image = image.to(device)
        target = target.long().to(device)
        
        optimizer.zero_grad()

        torch.use_deterministic_algorithms(False)
        logits = F.upsample(input=model(image), size=image.size()[2:4], mode='bilinear')
        logits = torch.movedim(logits, (0,1,2,3), (0,3,1,2))
        logits = logits.reshape((-1,15))
        target = target.reshape(-1)
        mask = target != -1
        logits = logits[mask]
        target = target[mask]
        loss = criterion(logits, target)
        #loss = loss.mean()
        loss = loss.sum() / weight[target].sum()
        loss.backward() 
        torch.use_deterministic_algorithms(True)
        
        training_batches += target.shape[0]

        training_loss.append((loss.data*target.shape[0]).tolist())
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), float('inf'))
        
        optimizer.step()
        
        model_ema.decay = ema_decay_schedule[it]
        model_ema.update(model)
        
        # Write running loss
        writer.add_scalar('training loss', loss , (epoch - 1) * len(train_loader)+i_board)
        i_board+=1
    
    logging.info("Training loss was: " + str(sum(training_loss) / training_batches))
    
    logging.info("Saving models")
    model_dir = os.path.join(up(os.path.abspath("/home/ubuntu/Tesi/Early-Fusion-Unet")), 'trained_models_EF_Unet')
    os.makedirs(model_dir, exist_ok=True)
    #torch.save(model.state_dict(), os.path.join(model_dir, 'model.pth'))
    
    # Start Evaluation
    if epoch % eval_every == 0 or epoch==1:
        model.eval()

        val_loss = []
        val_batches = 0
        y_true_val = []
        y_predicted_val = []
        
        seed_all(0)
        
        with torch.no_grad():
            for (image, target) in tqdm(val_loader, desc="validating"):
                
                image = image.to(device)
                target = target.to(device)

                #torch.use_deterministic_algorithms(False)
                logits = F.upsample(input=model(image), size=image.size()[2:4], mode='bilinear')
                logits = torch.movedim(logits, (0,1,2,3), (0,3,1,2))
                logits = logits.reshape((-1,15))
                target = target.reshape(-1)
                mask = target != -1
                logits = logits[mask]
                target = target[mask]
                loss = criterion(logits, target)
                #loss = loss.mean()
                loss = loss.sum() / weight[target].sum()
                #torch.use_deterministic_algorithms(True)
                
                probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
                target = target.cpu().numpy()
                
                val_batches += target.shape[0]
                val_loss.append((loss.data*target.shape[0]).tolist())
                y_predicted_val += probs.argmax(1).tolist()
                y_true_val += target.tolist()
                    
                
            y_predicted_val = np.asarray(y_predicted_val)
            y_true_val = np.asarray(y_true_val)
            #acc_val = Evaluation(y_predicted_val, y_true_val)
            
        val_loss_mean = sum(val_loss) / val_batches

        # Check if the current validation loss is the best encountered so far
        if val_loss_mean < best_val_loss:
            best_val_loss = val_loss_mean
            logging.info("Found new best validation loss at epoch {}: {}".format(epoch, best_val_loss))
            # Save the best model
            torch.save(model.state_dict(), os.path.join(model_dir, 'model.pth'))
            
        # Save Scores    
        logging.info("\n")
        logging.info("Evaluating model..")
        logging.info("Val loss was: " + str(sum(val_loss) / val_batches))
        logging.info("RESULTS AFTER EPOCH " +str(epoch) + ": \n")
        #logging.info("Evaluation: " + str(acc_val))
        
        writer.add_scalars('Loss per epoch', {'Val loss':sum(val_loss) / val_batches, 
                                                'Train loss':sum(training_loss) / training_batches}, epoch)
        
        #writer.add_scalar('Precision/val macroPrec', acc_val["macroPrec"] , epoch)
        #writer.add_scalar('Precision/val microPrec', acc_val["microPrec"] , epoch)
        #writer.add_scalar('Precision/val weightPrec', acc_val["weightPrec"] , epoch)
        #writer.add_scalar('Recall/val macroRec', acc_val["macroRec"] , epoch)
        #writer.add_scalar('Recall/val microRec', acc_val["microRec"] , epoch)
        #writer.add_scalar('Recall/val weightRec', acc_val["weightRec"] , epoch)
        #writer.add_scalar('F1/val macroF1', acc_val["macroF1"] , epoch)
        #writer.add_scalar('F1/val microF1', acc_val["microF1"] , epoch)
        #writer.add_scalar('F1/val weightF1', acc_val["weightF1"] , epoch)
        #writer.add_scalar('IoU/val MacroIoU', acc_val["IoU"] , epoch)        

        scheduler.step()
        model.train()

    # Start EMA Evaluation
    if (epoch % eval_every == 0 or epoch==1):
        
        logging.info("Saving models")
        model_dir = os.path.join(up(os.path.abspath("/home/ubuntu/Tesi/Early-Fusion-Unet")), 'trained_models_EF_Unet')
        os.makedirs(model_dir, exist_ok=True)
        #torch.save(model_ema.ema.state_dict(), os.path.join(model_dir, 'model_ema.pth'))

        val_loss_ema = []
        val_batches_ema = 0
        y_true_val_ema = []
        y_predicted_val_ema = []
                        
        seed_all(0)
        
        with torch.no_grad():
            for (image, target) in tqdm(val_loader, desc="validating"):
                
                image = image.to(device)
                target = target.to(device)

                logits = model_ema.ema(image)
                #torch.use_deterministic_algorithms(False)
                logits = F.upsample(input=logits, size=image.size()[2:4], mode='bilinear')
                logits = torch.movedim(logits, (0,1,2,3), (0,3,1,2))
                logits = logits.reshape((-1,15))
                target = target.reshape(-1)
                mask = target != -1
                logits = logits[mask]
                target = target[mask]
                loss = criterion(logits, target) # Calcolo della loss
                #loss = loss.mean()
                loss = loss.sum() / weight[target].sum()
                #torch.use_deterministic_algorithms(True)
                
                probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
                target = target.cpu().numpy()
                
                val_batches_ema += target.shape[0]
                val_loss_ema.append((loss.data*target.shape[0]).tolist())
                y_predicted_val_ema += probs.argmax(1).tolist()
                y_true_val_ema += target.tolist()
                
            y_predicted_val_ema = np.asarray(y_predicted_val_ema)
            y_true_val_ema = np.asarray(y_true_val_ema)
            #acc_val_ema = Evaluation(y_predicted_val_ema, y_true_val_ema)
            
        val_loss_mean = sum(val_loss_ema) / val_batches_ema

        # Check if the current validation loss is the best encountered so far
        if val_loss_mean < best_val_loss_ema:
            best_val_loss_ema = val_loss_mean
            logging.info("Found new best validation loss at epoch {}: {}".format(epoch, best_val_loss_ema))
            # Save the best model
            torch.save(model.state_dict(), os.path.join(model_dir, 'model_ema.pth'))
            
        # Save Scores
        logging.info("\n")
        logging.info("Evaluating EMA model..")
        logging.info("val loss was: " + str(sum(val_loss_ema) / val_batches_ema))
        logging.info("RESULTS AFTER EPOCH " +str(epoch) + ": \n")
        #logging.info("Evaluation: " + str(acc_val_ema))
        
        writer.add_scalars('Loss per epoch (EMA)', {'val loss':sum(val_loss_ema) / val_batches_ema}, epoch)
        #writer.add_scalar('Precision/val macroPrec (EMA)', acc_val_ema["macroPrec"] , epoch)
        #writer.add_scalar('Precision/val microPrec (EMA)', acc_val_ema["microPrec"] , epoch)
        #writer.add_scalar('Precision/val weightPrec (EMA)', acc_val_ema["weightPrec"] , epoch)
        #writer.add_scalar('Recall/val macroRec (EMA)', acc_val_ema["macroRec"] , epoch)
        #writer.add_scalar('Recall/val microRec (EMA)', acc_val_ema["microRec"] , epoch)
        #writer.add_scalar('Recall/val weightRec (EMA)', acc_val_ema["weightRec"] , epoch)
        #writer.add_scalar('F1/val macroF1 (EMA)', acc_val_ema["macroF1"] , epoch)
        #writer.add_scalar('F1/val microF1 (EMA)', acc_val_ema["microF1"] , epoch)
        #writer.add_scalar('F1/val weightF1 (EMA)', acc_val_ema["weightF1"] , epoch)
        #writer.add_scalar('IoU/val MacroIoU (EMA)', acc_val_ema["IoU"] , epoch)

training:   0%|          | 0/286 [00:00<?, ?it/s]/home/ubuntu/.local/lib/python3.8/site-packages/torch/nn/functional.py:3509: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/ubuntu/.local/lib/python3.8/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
validating: 100%|██████████| 129/129 [00:12<00:00, 10.37it/s]


In [17]:
torch.save(model_ema.ema.state_dict(), os.path.join(model_dir, 'model_ema_80.pth'))
torch.save(model.state_dict(), os.path.join(model_dir, 'model_80.pth'))

Test

In [18]:
models_list2 = []
#models_files = glob(os.path.join(os.path.join("C:\\Users\\lucap\\OneDrive\\Desktop\\Tesi\\Python Code\\trained_models_finito", '1'),'*.pth'))
models_files = glob(os.path.join(os.path.join("/home/ubuntu/Tesi/trained_models_EF_Unet"),'model_ema_80.pth'))
#models_files = glob("C:\Users\lucap\OneDrive\Desktop\Tesi\trained_models\10\model_ema.pth")

for model_file in models_files:

    #model2 = MariNext(11, 15)
    model2 = smp.Unet('resnet18', in_channels=11, classes=15)
    torch.nn.init.xavier_uniform_(model2.encoder.conv1.weight)

    model2.to(device)

    # Load model from specific epoch to continue the training or start the evaluation
    
    logging.info('Loading model files from folder: %s' % model_file)

    checkpoint = torch.load(model_file, map_location = device)
    checkpoint = {k.replace('decode_head','decoder'):v for k,v in checkpoint.items() if ('proj1' not in k) and ('proj2' not in k)}

    model2.load_state_dict(checkpoint)

    del checkpoint  # dereference
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    model2.eval()
    
    models_list2.append(model2)

In [ ]:
y_true = []
y_predicted = []
                            
with torch.no_grad():
    for (image, target) in tqdm(test_loader, desc="testing"):

        image = TTA(image)            

        image = image.to(device)
        target = target.to(device)
        
        logits = model2(image)
        logits = F.upsample(input=logits, size=(target.shape[-2], target.shape[-1]), mode='bilinear')
            
        # Accuracy metrics only on annotated pixels
        probs = torch.nn.functional.softmax(logits, dim=1)
        predictions = probs.argmax(1)
        
        predictions = TTA(predictions, reverse_aggregation = True)
            
        predictions = predictions.reshape(-1)
        target = target[0].reshape(-1)
        mask = target != -1
        
        predictions = predictions[mask].cpu().numpy()
        target = target[mask]
        
        target = target.cpu().numpy()

        y_predicted += predictions.tolist()
        y_true += target.tolist()

    # Save Scores
    #acc = Evaluation(y_predicted, y_true)
    #logging.info("\n")
    #logging.info("STATISTICS: \n")
    #logging.info("Evaluation: " + str(acc))
    #print("Evaluation: " + str(acc))
    conf_mat = confusion_matrix(y_true, y_predicted, labels, True)
    logging.info("Confusion Matrix:  \n" + str(conf_mat.to_string()))
    print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
